In [11]:
#!/usr/bin/env python3

import pandas as pd
import os

def merge_all_csvs_no_unify(schedule_csv, stats_files, output_csv):
    """
    1) Load the schedule CSV (one row per match).
       - rename match_report -> match_id if needed
       - final_df = schedule
    2) For each stats CSV:
       - rename match_report -> match_id if needed
       - group by match_id, keeping the first row => exactly 1 row per match
       - merge with final_df on match_id (left join)
         using a unique suffix so that overlapping columns remain separate
    3) Save final_df, which has exactly one row per match from the schedule,
       plus all columns from each CSV (with suffixes for collisions).
    """
    print(f"[INFO] Loading schedule: {schedule_csv}")
    schedule_df = pd.read_csv(schedule_csv)
    
    # If schedule has 'match_report' but no 'match_id', rename
    if "match_report" in schedule_df.columns and "match_id" not in schedule_df.columns:
        schedule_df.rename(columns={"match_report": "match_id"}, inplace=True)

    if "match_id" not in schedule_df.columns:
        raise ValueError("Schedule file needs a 'match_id' (or 'match_report') column.")

    schedule_df["match_id"] = schedule_df["match_id"].astype(str)
    
    # final_df starts as the schedule
    final_df = schedule_df.copy()

    # Merge each stats file
    for i, path in enumerate(stats_files, start=1):
        if not path:
            continue
        print(f"[INFO] Processing stats #{i}: {path}")
        try:
            df_stats = pd.read_csv(path)
        except Exception as e:
            print(f"    [ERROR] Could not read '{path}': {e}")
            continue
        
        # rename match_report -> match_id if needed
        if "match_report" in df_stats.columns and "match_id" not in df_stats.columns:
            df_stats.rename(columns={"match_report": "match_id"}, inplace=True)

        if "match_id" not in df_stats.columns:
            print(f"    [WARNING] No 'match_id' in {path}; skipping.")
            continue
        
        df_stats["match_id"] = df_stats["match_id"].astype(str)

        # Group by match_id so there's exactly one row per match
        # e.g. if stats CSV has multiple lines for the same match, we keep the first
        df_stats_1row = df_stats.groupby("match_id", as_index=False).first()

        # Merge onto final_df
        # We'll use a custom suffix to keep all columns separate if there's a name collision
        # e.g. suffix = "_sX" where X is i, so the second file gets _s2, etc.
        suffix_str = f"_s{i}"
        merged = pd.merge(
            final_df,
            df_stats_1row,
            on="match_id",
            how="left",
            suffixes=("", suffix_str)  # keep collisions with a unique suffix
        )

        # We do NOT unify or combine collisions. We simply let them remain.
        # If final_df had "date", and df_stats also has "date", we'll get "date" and "date_s1".
        final_df = merged
    
    # Save final
    print(f"[INFO] Final DataFrame has {len(final_df)} rows (one row per match). Saving to {output_csv}")
    final_df.to_csv(output_csv, index=False)
    print("[INFO] Done!")

def main():
    schedule_csv = "ENG-Premier League_schedule.csv"
    stats_files = [
        "eng-premier_league_defense_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "eng-premier_league_goal_shot_creation_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "eng-premier_league_keeper_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "eng-premier_league_misc_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "eng-premier_league_passing_types_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "eng-premier_league_possession_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "eng-premier_league_shooting_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
    ]
    output_csv = "unified_premier_league.csv"
    merge_all_csvs_no_unify(schedule_csv, stats_files, output_csv)
    
    schedule_csv = "ESP-La Liga_schedule.csv"
    output_csv = "unified_la_liga.csv"
    stats_files = [
        "esp-la_liga_defense_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "esp-la_liga_goal_shot_creation_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "esp-la_liga_keeper_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "esp-la_liga_passing_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "esp-la_liga_passing_types_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "esp-la_liga_possession_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv",
        "esp-la_liga_shooting_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv"
    ]
    merge_all_csvs_no_unify(schedule_csv, stats_files, output_csv)
if __name__ == "__main__":
    main()


[INFO] Loading schedule: ENG-Premier League_schedule.csv
[INFO] Processing stats #1: eng-premier_league_defense_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Processing stats #2: eng-premier_league_goal_shot_creation_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Processing stats #3: eng-premier_league_keeper_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Processing stats #4: eng-premier_league_misc_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Processing stats #5: eng-premier_league_passing_types_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Processing stats #6: eng-premier_league_possession_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Processing stats #7: eng-premier_league_shooting_['2020-2021',_'2021-2022',_'2022-2023',_'2023-2024',_'2024-2025'].csv
[INFO] Final DataFrame has 1900 rows (one row per m